# json파일 전처리

In [3]:
import json
import pandas as pd

json_data = "C:\\Users\\IMS\Desktop\\Hwangsihoon\\WebCam\\participant_data.json"

with open(json_data, 'r') as f:
    data = json.load(f)

# user_name 초기화를 위한 빈 리스트 생성
records = []

# 각 user_name에 대해 meta 정보와 dot_info 정보 추출 및 데이터프레임 생성
for user_name, user_data in data.items():
    meta_info = user_data["meta"]
    dot_info = user_data["dot_info"]
    for n_key, n_value in dot_info.items():
        for m_key, m_value in n_value.items():
            # 각 dot_info 항목에 meta 정보 및 user_name 추가, n, m_key : 파일명 -> User_n_m
            combined_data = {**meta_info, **m_value, "user_name": user_name, "n_key": n_key, "m_key": m_key}
            records.append(combined_data)

df = pd.DataFrame(records)

# 왼쪽, 오른쪽 눈 df화
df_right = df[['user_name', 'file_name_right', 'right_landmarks', 'label']]
df_left = df[['user_name', 'file_name_left', 'left_landmarks', 'label']]


In [4]:
import os
import glob
from keras.preprocessing.image import load_img, img_to_array

import numpy as np

folder_path = "C:\\Users\\IMS\\Desktop\\Hwangsihoon\\WebCam\img"

files = glob.glob(os.path.join(folder_path, '*.jpg'))

# right파일, left파일 각각 분리
right_images = [file for file in files if 'right' in os.path.basename(file).lower()]
left_images = [file for file in files if 'left' in os.path.basename(file).lower()]


def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

right_images = [load_and_preprocess_image(img_path) for img_path in right_images]
left_images = [load_and_preprocess_image(img_path) for img_path in left_images]

right_labels = np.array(df_right['label'].tolist())
left_labels = np.array(df_left['label'].tolist())

print(right_labels.shape)
print(type(right_labels))

(31995, 2)
<class 'numpy.ndarray'>


In [5]:
from sklearn.model_selection import train_test_split

# 데이터셋을 훈련과 테스트로 분할
right_images_train, right_images_test, left_images_train, left_images_test, right_labels_train, right_labels_test = train_test_split(
    right_images, left_images, right_labels, test_size=0.4, shuffle=True, random_state=42
)

# CNN Model

In [6]:
import tensorflow as tf
from keras import layers, Model, Input


x1 = right_images  # 이미지
x2 = left_images

y1 = right_labels  # 레이블
y2 = left_labels

# input1 모델
input1 = Input(shape=(128, 128, 3))
x1 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(input1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)
x1 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)
x1 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)
x1 = layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(x1)
x1 = layers.Flatten()(x1)  # Flatten 레이어
x1 = layers.Dense(128, activation='relu')(x1)

# input2 모델
input2 = Input(shape=(128, 128, 3))
x2 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(input2)
x2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)
x2 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(x2)
x2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)
x2 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x2)
x2 = layers.MaxPooling2D(pool_size=(2, 2))(x2)
x2 = layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(x2)
x2 = layers.Flatten()(x2)  # Flatten 레이어
x2 = layers.Dense(128, activation='relu')(x2)

# 직렬 Concatenate
combined = layers.Concatenate()([x1, x2])

# Fully Connected 층
fc_output = layers.Dense(8, activation='relu')(combined)
final_output = layers.Dense(2, activation=None)(fc_output)

model = Model(inputs=[input1, input2], outputs=final_output)

# 모델 컴파일 및 요약
model.compile(optimizer='adam', loss='mse')
model.summary()



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 128, 128, 32) 896         input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error
import tensorflow as tf
import numpy as np

right_images_train = np.array(right_images_train)
left_images_train = np.array(left_images_train)
right_labels_train = np.array(right_labels_train)
# left_labels_train = np.array(left_labels_train)

right_images_test = np.array(right_images_test)
left_images_test = np.array(left_images_test)
right_labels_test = np.array(right_labels_test)
# left_labels_test = np.array(left_labels_test)


# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics=[tf.keras.metrics.MeanSquaredLogarithmicError()])

# 모델 학습
history = model.fit(
    [right_images_train, left_images_train],  # 입력 데이터 리스트
    right_labels_train,                       # 레이블
    epochs=40,                                # 에포크 수
    batch_size=32,                          # 배치 사이즈
    validation_data=([right_images_test, left_images_test], right_labels_test),  # 검증 데이터와 레이블
    verbose=2                                
)

# 모델 평가
loss, msle_metric = model.evaluate([right_images_test, left_images_test], right_labels_test, verbose=0)
print(f'Test Loss (MSLE): {loss}')
print(f'Test MSLE (metric): {msle_metric}')

# 예측
predictions = model.predict([right_images_test, left_images_test])

# 평가 지표 계산
mse = mean_squared_error(right_labels_test, predictions)
mae = mean_absolute_error(right_labels_test, predictions)
msle = mean_squared_log_error(right_labels_test, predictions)
r2 = r2_score(right_labels_test, predictions)

# 결과 출력
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Logarithmic Error: {msle}')
print(f'R² Score: {r2}')

# 오차 계산
errors = right_labels_test - predictions

# 예측 값, 실제 값 및 오차 출력 (예시로 첫 5개 데이터만 출력)
for i in range(5):  # 예시로 첫 5개 데이터만 출력
    print(f'Actual Value:\n{right_labels_test[i]}')
    print(f'Predicted Value:\n{predictions[i]}')
    print(f'Error:\n{errors[i]}\n')